# AI Applications - Mini Project 2
> By Oliver Dietsche & Simon Peier

## Imports

In [4]:
import os
import tensorflow as tf
import numpy as np

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from PIL import Image

## Dataset
We decided to use part of the [Tiny ImageNet](https://www.kaggle.com/c/tiny-imagenet) dataset. 

According to the description in Kaggle, the Tiny ImageNet dataset has the following properties: "Tiny ImageNet contains 200 classes for training. Each class has 500 images. The test set contains 10,000 images. All images are 64x64 colored ones." It fulfills all the required key characteristics, except that it has too many classes (and thus too many samples). To meet all criteria, we pre-processed the data-set and chose 6 classes to use. As the dataset contains lots of classes of different categories, we settled for classes from the animal realm.

### Dataset import

In [6]:
data_dir = "dataset"
words_file = os.path.join(data_dir, "words.txt")
image_size = (64, 64)

# Load words.txt
with open(words_file, 'r') as f:
    class_labels = {}
    for line in f:
        line = line.strip().split('\t')
        class_labels[line[0]] = line[1]

# Load images and labels
images = []
labels = []

for folder_name in os.listdir(data_dir):
    if folder_name.startswith('n'):
        label = class_labels[folder_name]
        image_folder_path = os.path.join(data_dir, folder_name, "images")
        for image_name in os.listdir(image_folder_path):
            image_path = os.path.join(image_folder_path, image_name)
            image = img_to_array(load_img(image_path))
            images.append(image)
            labels.append(label)

images = np.array(images)
labels = np.array(labels)


### Dataset split

In [12]:
# split into test and training data
train_val_imgs, test_imgs, train_val_labels, test_labels = train_test_split(images, labels, test_size=0.3)

In [13]:
# split into training and validation data
train_data, val_data, train_labels, val_labels = train_test_split(train_val_imgs, train_val_labels, test_size=0.2)

### Visualization

In [14]:
# TODO